# Réception, décodage et affichage d’une image satellite METEOR-M2
                               PARTIE 2: Décompression JPEG et affichage de l'image

Dans la partie précédente nous avons réceptionné le signal provenant du satellite météorologique
Meteor-M2, puis nous avons traité ce signal afin de pouvoir récupérer les données images compressées
au format JPEG. L’objectif de cette partie est de décoder plusieurs imagettes de 8x8 pixels et de
vérifier que le décodage correspond bien à une partie de l’image complète décodée par le programme
"meteor-decoder".


# I. EXTRACTION DES DONNÉES UTILES

La fonction extract(fichier) contenue dans "extract_viterbi.py" effectue les actions suivantes :

— Ouvre le fichier binaire issue du décodage précédent (viterbi).

— Réalise la conversion en hexadécimal.

— Extrait tous les octets compris entre deux mots de synchronisation (1acffc1d), réalise la conversion en
décimal et place chaque ligne dans un array en forme de tableau.

— Pour chaque ligne, il affiche l’endroit à partir duquel commencent les informations à exploiter.

Toutes ces étapes sont visibles dans le fichier "extract_viterbi.py" et ne présentent pas de difficultés
particulières. Elles ont été réalisées en suivant les explications de [2] p6-8.

1. **Effectuer les opérations ci-dessus en important le fichier :**


In [3]:
import extract_viterbi


ligne 0
[ 64  36 202   0 151   0   0   2 153 251 232   0   0  28   0   0 255 240  78 230 116 171 178 146 180  72   7 151  32  32 253  69  87 138 222  57  53  48  67 224 111  25 200 174 151]
ligne 1
[ 64  36 207   1  21   0   0   2 153 251 232   0   0  98   0   0 255 240  86 228  53 141  82  45  78 217 231 242 176 241  16  73  71  35  56 245  21 206  45 204 211  70 145 142 173]
ligne 2
[ 64  36 210   0 201   0   0   2 153 251 232   0   0 140   0   0 255 240  86 242 113  13 193 158  20  92 102 108 170  30   6 238 113 158 188  86 156 122  19 164 228 220 200 145 196]
ligne 3
[ 65 100 214   1  31   0   0   2 153 251 232   0   0   0   0   0 255 240  88 242  57 124  67 105 101 172 233 179  93  68 102 178 142  65  52 130  60   6 104 199  81 245  24 174 110]
ligne 4
[ 65  36 217   0 207   0   0   2 153 251 232   0   0  42   0   0 255 240  88 241 123 131   5 173 243 219 201  46 220 182  64  10 122 127 144 106  69 202  21 104 229 238   8  24 206]
ligne 5
[ 65  36 220   1 107   0   0   2 153 251 2

Si vous souhaitez changer le fichier à traiter, ouvrir le fichier extract_viterbi.py et modifier l’instruction (autour de la ligne 100) :

extract_viterbi.extraction ("viterbi2.bin")


In [2]:
#import importlib
#importlib.reload(extract_viterbi) # A refaire apres chaque modification du fichier


![title](images/viterbi2.png)

Quasiment toutes les lignes commencent par la valeur 64, 65 ou 68. Ce sont les APID (Packet
identifier) de l’instrument à bord qui transmet les données (On rappelle qu’il y a bien trois gammes de
longueurs d’onde). De temps en temps, une ligne commence par la valeur 70. Il s’agit d’une information
de télémétrie.

2. **Identifier une ligne de télémétrie et trouver l’heure envoyée par le satellite à partir de
l’octet 21 (en comptant l’octet 70 comme 0). Est-ce cohérent avec l’information donnée
par meteor-decoder : "Onboard time : 12 :07 :25.928". Notre décodage est-il validé jusque-là ?**


ligne 10: 12 7 25 .... ok

Nous allons nous intéresser dans la suite à la ligne 26, concernant des données provenant de
l’instrument ayant pour APID 65. Sur une telle ligne, l’octet 18 correspond au facteur de qualité
Q (qui nous servira plus tard) et les octets suivants concernent les données de l’image envoyée.

3. **Identifier la ligne en question, donner la valeur du facteur de qualité Q pour cette ligne
ainsi que les huit premiers octets correspondants aux données de l’image.**


ligne 26: Q=96

248  91  88 248   7 246  31 216  sont les 8 premiers octets correspondants aux données de l'image.

# II. DÉCODAGE DE HUFFMAN
Commençons par revoir et approfondir un peu le principe de la compression JPEG :

![title](images/jpeg.png)
![title](images/jpeg2.png)
![title](images/jpeg3.png)
![title](images/jpeg4.png)
4. **Décoder les 7 premiers octets des données de la ligne 26 sachant qu’ils sont encodés selon
l’algorithme de Huffman (commencer par les convertir en binaire sur 8 bits).**


Les données sont : 248, 91, 88 , 248, 7, 246, 31, 216 ... 

Ce qui donne en binaire sur 8 bits :

11111000 01011011 01011000 11111000 00000111 11110110 00011111...

Premier code DC possible : 111110 → 8 bits suivants : 00010110 → 22 − (2 8 − 1) = -233

On continue avec 11 01011000 11111000 00000111 11110110 00011111

Premier code AC 11010 → code 0/5 : pas de zéro avant et on lit les 5 bits suivants : 11000 = 24

Il reste : 11111000 00000111 11110110 00011111

Code AC : 11111000 → code 0/7 : pas de zéro et on lit les 7 bits suivants : 0000011 → 3 − (2 7 − 1) =
-124

Il reste 1 11110110 00011111

Code AC : 111110110 → code 1/4 : un zéro et on lit les 4 bits suivants : 0001 → 1 − (2 4 − 1) = -14

On n’oublie pas le 0 avant le -14 et pour l’instant nous obtenons :

[-233, 24, -124, 0, -14...]

5. **Vérifier que votre décodage est correct en utilisant la fonction ci-dessous. Pourquoi a
t-on ajouté 1010 à la fin ?**


In [4]:
 extract_viterbi.huffman("1111100001011011010110001111100000000111111101100001" + "1010")


dct= [-233, 24, -124, 0, -14]




"1010" EOB pour arrêter le décodage

6. **Réaliser le décodage de la ligne 26 complète sachant qu’elle contient un MCU complet
de 14 imagettes en utilisant la fonction :**


In [5]:
extract_viterbi.huff(26,14)


octets= [248  91  88 248   7 246  31 216 219 196 115 232 115  89 219 248 175 195 151 145 234 205  52 209  43  92  40 134  54 139 106  35  28 237  69 105  47  25 136 218 139 111 176   1 230   7 168  53  95 217 243 246 128 248 163 162 252  56 139 226  79 197 184 254  34 233  55  51 201 118 116 171 219  51  45 189 165 218 219 198  17]

nbr total octets compressés= 740
dct= [-233, 24, -124, 0, -14, -201, 35, 7, -28, 6, -12, -8, 43, -35, 12, -4, 22, 2, 6, 6, -5, -2, 2, 8, -2, -1, -6, -4, 2, -5, 0, -2, -3, -3, 2, 1, 0, 1, 0, -2, -3, -2, 2, 4, 12, 2, 2, 1, -1, 0, -2, -3, -2, -8, 0, -3, -1, -1, -1, 0, 4, -1, 3]
2
dct= [-7, 21, -131, 128, 81, -26, -71, -5, 73, 91, -4, 69, -22, 9, 7, 4, 11, 1, -21, 13, -12, 1, 5, -8, -11, 11, -2, -8, -4, -1, -3, -2, -2, 3, -5, 3, -1, -2, 0, 0, 3, -1, 7, 10, -4, -1, 2, -1, 1, -2, -1, 0, -1, -3, -3, 1, -2, 1, 0, 0, 3, -1, 1]
3
dct= [111, -43, -102, -61, 50, -12, 3, -26, 27, -88, -29, -52, -5, -46, -1, 7, -37, 6, -5, -14, 19, 1, 3, 0, 10, 11, -4, 9, 4, -2, 6, 9, -2

[[],
 [-233,
  24,
  -124,
  0,
  -14,
  -201,
  35,
  7,
  -28,
  6,
  -12,
  -8,
  43,
  -35,
  12,
  -4,
  22,
  2,
  6,
  6,
  -5,
  -2,
  2,
  8,
  -2,
  -1,
  -6,
  -4,
  2,
  -5,
  0,
  -2,
  -3,
  -3,
  2,
  1,
  0,
  1,
  0,
  -2,
  -3,
  -2,
  2,
  4,
  12,
  2,
  2,
  1,
  -1,
  0,
  -2,
  -3,
  -2,
  -8,
  0,
  -3,
  -1,
  -1,
  -1,
  0,
  4,
  -1,
  3],
 [-226,
  21,
  -131,
  128,
  81,
  -26,
  -71,
  -5,
  73,
  91,
  -4,
  69,
  -22,
  9,
  7,
  4,
  11,
  1,
  -21,
  13,
  -12,
  1,
  5,
  -8,
  -11,
  11,
  -2,
  -8,
  -4,
  -1,
  -3,
  -2,
  -2,
  3,
  -5,
  3,
  -1,
  -2,
  0,
  0,
  3,
  -1,
  7,
  10,
  -4,
  -1,
  2,
  -1,
  1,
  -2,
  -1,
  0,
  -1,
  -3,
  -3,
  1,
  -2,
  1,
  0,
  0,
  3,
  -1,
  1],
 [-337,
  -43,
  -102,
  -61,
  50,
  -12,
  3,
  -26,
  27,
  -88,
  -29,
  -52,
  -5,
  -46,
  -1,
  7,
  -37,
  6,
  -5,
  -14,
  19,
  1,
  3,
  0,
  10,
  11,
  -4,
  9,
  4,
  -2,
  6,
  9,
  -2,
  5,
  -4,
  1,
  -2,
  6,
  2,
  1,
  -1,
  4,
  -6,
  -2,


où line correspond à la ligne souhaitée et nbr au nombre d’imagettes à décoder.
Lors du décodage manuel, on se rend compte que certains octets sont codés sur plus de huit bits,
étrange pour une méthode de compression ? Pour l’exemple traité à la main plus haut, le gain en
compression n’est pas flagrant... Qu’en est-il lorsqu’on considère un plus grand nombre d’octets ?
7. **La fonction précédente affiche dans la console le nombre d’octets compressés et le nombre
d’octets décompressés. Calculer le pourcentage de compression gagné avec la méthode de
Huffman. Commenter.**


On remarque pour la ligne 26 : nbr total octets compressés= 740 et nbr total octets decompressés=
874. Il y a bien un gain de place : (874 − 740)/874 = 0,15 donc environ 15% de gagné grâce à la
compression de Huffman. En effet, statistiquement, il y a plus de code < 8 bits que de code > 8 bits.
Le RLE sur la valeur 0 permet également de gagner beaucoup de place.

8. **Calculer le taux de compression associé à cette zone (on ne tient pas compte des tables
à joindre au fichier ni du facteur de qualité à transmettre également). Commenter.**


On a 14x8x8 = 896 pixels à afficher. Sur un canal, un pixel est codé par un octet donc une taille
finale de 896 octets. On a vu précédemment qu’il y avait 740 octets compressés. taux compression =
896/740 = 1,21. Il s’agit d’une zone faiblement compressée. L’essentiel de la compression est ici effectué
par le codage de Huffman.

# III. DES ZIGS ET DES ZAGS
9. **Compléter les 15 premières valeurs de la première matrice de la ligne 26 en utilisant la
lecture en zigzag :**


-233 ... ... ... ... 

...  ... ... ... ...

... ... ... ... ...

... ... ...

... ...

...

Puis vérifier avec la fonction :


In [6]:
extract_viterbi.zigzag(26,14,0)


octets= [248  91  88 248   7 246  31 216 219 196 115 232 115  89 219 248 175 195 151 145 234 205  52 209  43  92  40 134  54 139 106  35  28 237  69 105  47  25 136 218 139 111 176   1 230   7 168  53  95 217 243 246 128 248 163 162 252  56 139 226  79 197 184 254  34 233  55  51 201 118 116 171 219  51  45 189 165 218 219 198  17]

nbr total octets compressés= 740
dct= [-233, 24, -124, 0, -14, -201, 35, 7, -28, 6, -12, -8, 43, -35, 12, -4, 22, 2, 6, 6, -5, -2, 2, 8, -2, -1, -6, -4, 2, -5, 0, -2, -3, -3, 2, 1, 0, 1, 0, -2, -3, -2, 2, 4, 12, 2, 2, 1, -1, 0, -2, -3, -2, -8, 0, -3, -1, -1, -1, 0, 4, -1, 3]
2
dct= [-7, 21, -131, 128, 81, -26, -71, -5, 73, 91, -4, 69, -22, 9, 7, 4, 11, 1, -21, 13, -12, 1, 5, -8, -11, 11, -2, -8, -4, -1, -3, -2, -2, 3, -5, 3, -1, -2, 0, 0, 3, -1, 7, 10, -4, -1, 2, -1, 1, -2, -1, 0, -1, -3, -3, 1, -2, 1, 0, 0, 3, -1, 1]
3
dct= [111, -43, -102, -61, 50, -12, 3, -26, 27, -88, -29, -52, -5, -46, -1, 7, -37, 6, -5, -14, 19, 1, 3, 0, 10, 11, -4, 9, 4, -2, 6, 9, -2

array([[-233.,   24., -201.,   35.,   12.,   -4.,   -4.,    2.],
       [-124.,  -14.,    7.,  -35.,   22.,   -6.,   -5.,    2.],
       [   0.,  -28.,   43.,    2.,   -1.,    0.,   -2.,    4.],
       [   6.,   -8.,    6.,   -2.,   -2.,   -3.,   12.,   -8.],
       [ -12.,    6.,    8.,   -3.,   -2.,    2.,   -2.,    0.],
       [  -5.,    2.,   -3.,    0.,    2.,   -3.,   -3.,    4.],
       [  -2.,    2.,    1.,    1.,   -2.,   -1.,    0.,   -1.],
       [   1.,    0.,   -1.,    0.,   -1.,   -1.,    3.,    0.]])

ligne : le numéro de la ligne,
nbr_total : le nombre d’imagettes dans la ligne considérée,
matrice : le numéro de la matrice à afficher

10. **Faire afficher la matrice suivante. En vérifiant avec la liste affichée plus haut dans la
console, que constate-t-on pour la valeur DC ? Prédire la valeur DC de la troisième
matrice.**


In [16]:
extract_viterbi.zigzag(26,14,1)


octets= [248  91  88 248   7 246  31 216 219 196 115 232 115  89 219 248 175 195 151 145 234 205  52 209  43  92  40 134  54 139 106  35  28 237  69 105  47  25 136 218 139 111 176   1 230   7 168  53  95 217 243 246 128 248 163 162 252  56 139 226  79 197 184 254  34 233  55  51 201 118 116 171 219  51  45 189 165 218 219 198  17]

nbr total octets compressés= 740
dct= [-233, 24, -124, 0, -14, -201, 35, 7, -28, 6, -12, -8, 43, -35, 12, -4, 22, 2, 6, 6, -5, -2, 2, 8, -2, -1, -6, -4, 2, -5, 0, -2, -3, -3, 2, 1, 0, 1, 0, -2, -3, -2, 2, 4, 12, 2, 2, 1, -1, 0, -2, -3, -2, -8, 0, -3, -1, -1, -1, 0, 4, -1, 3]
2
dct= [-7, 21, -131, 128, 81, -26, -71, -5, 73, 91, -4, 69, -22, 9, 7, 4, 11, 1, -21, 13, -12, 1, 5, -8, -11, 11, -2, -8, -4, -1, -3, -2, -2, 3, -5, 3, -1, -2, 0, 0, 3, -1, 7, 10, -4, -1, 2, -1, 1, -2, -1, 0, -1, -3, -3, 1, -2, 1, 0, 0, 3, -1, 1]
3
dct= [111, -43, -102, -61, 50, -12, 3, -26, 27, -88, -29, -52, -5, -46, -1, 7, -37, 6, -5, -14, 19, 1, 3, 0, 10, 11, -4, 9, 4, -2, 6, 9, -2

array([[-226.,   21.,  -26.,  -71.,    7.,    4.,   -8.,   -4.],
       [-131.,   81.,   -5.,    9.,   11.,   -2.,   -1.,    7.],
       [ 128.,   73.,  -22.,    1.,   11.,   -3.,   -1.,   10.],
       [  91.,   69.,  -21.,  -11.,   -2.,    3.,   -4.,   -3.],
       [  -4.,   13.,   -8.,   -2.,    0.,   -1.,   -1.,   -3.],
       [ -12.,    5.,    3.,    0.,    2.,    0.,    1.,    3.],
       [   1.,   -5.,   -2.,   -1.,   -1.,   -2.,    0.,   -1.],
       [   3.,   -1.,    1.,   -2.,    1.,    0.,    1.,    0.]])

Dans la liste calculée plus haut, on a [-7, 21, -131, 128, 81 ...etc.]. La valeur DC est de -7 et pas
-226. 

En effet, d’une imagette à l’autre la valeur DC indiquée est relative à celle précédente : -233 -
(-7) = -226. Les valeurs AC sont inchangées.

Pour la troisième matrice, on a dans la liste DC=111 donc -226 - 111 = -337

# IV. QUANTIFICATION INVERSE
Commencer par créér une variable DCT qui va stocker la valeur de la première matrice en zigzag :


In [7]:
DCT = extract_viterbi.zigzag(26,14,0)


octets= [248  91  88 248   7 246  31 216 219 196 115 232 115  89 219 248 175 195 151 145 234 205  52 209  43  92  40 134  54 139 106  35  28 237  69 105  47  25 136 218 139 111 176   1 230   7 168  53  95 217 243 246 128 248 163 162 252  56 139 226  79 197 184 254  34 233  55  51 201 118 116 171 219  51  45 189 165 218 219 198  17]

nbr total octets compressés= 740
dct= [-233, 24, -124, 0, -14, -201, 35, 7, -28, 6, -12, -8, 43, -35, 12, -4, 22, 2, 6, 6, -5, -2, 2, 8, -2, -1, -6, -4, 2, -5, 0, -2, -3, -3, 2, 1, 0, 1, 0, -2, -3, -2, 2, 4, 12, 2, 2, 1, -1, 0, -2, -3, -2, -8, 0, -3, -1, -1, -1, 0, 4, -1, 3]
2
dct= [-7, 21, -131, 128, 81, -26, -71, -5, 73, 91, -4, 69, -22, 9, 7, 4, 11, 1, -21, 13, -12, 1, 5, -8, -11, 11, -2, -8, -4, -1, -3, -2, -2, 3, -5, 3, -1, -2, 0, 0, 3, -1, 7, 10, -4, -1, 2, -1, 1, -2, -1, 0, -1, -3, -3, 1, -2, 1, 0, 0, 3, -1, 1]
3
dct= [111, -43, -102, -61, 50, -12, 3, -26, 27, -88, -29, -52, -5, -46, -1, 7, -37, 6, -5, -14, 19, 1, 3, 0, 10, 11, -4, 9, 4, -2, 6, 9, -2

Pour la quantification on utilise une matrice classique de quantification que l’on nomme HTK (voir
ci-dessous)
Cette matrice est modifiée en fonction du facteur de qualité Q selon les instructions suivantes :

— Si Q est compris entre 20 et 50 : F = 5000 / Q
 et F = 200 − 2 × Q sinon.

— On en déduit une nouvelle matrice PTK dont les éléments sont ceux de HTK multiplié par F/100.

— La matrice originale avant quantification F0 est obtenue en multipliant élément par élément DCT
et PTK.

11. **Ouvrir le fichier extract_viterbi.py et compléter la fonction quantif(DCT,Q) (vers les
lignes 520) en tenant compte des instructions ci-dessus (cf Figure 7).**

![title](images/fig7.png)

12. **Executer la fonction pour afficher la matrice F0 et la stocker dans une variable F0 :**

En ayant bien pris soin de remplacer la valeur de Q par celle trouvée à la question 18.



In [8]:
import importlib # Faire qu’une fois
import extract_viterbi2
importlib.reload(extract_viterbi2)   # Refaire apres chaque modification
F0 = extract_viterbi.quantif(DCT,96)


[ 64   1 222 244  25  97 193 161 186 197  72 119  53  11 115]
PTK= [[ 1  1  1  1  2  3  4  5]
 [ 1  1  1  2  2  5  5  4]
 [ 1  1  1  2  3  5  6  4]
 [ 1  1  2  2  4  7  6  5]
 [ 1  2  3  4  5  9  8  6]
 [ 2  3  4  5  6  8  9  7]
 [ 4  5  6  7  8 10 10  8]
 [ 6  7  8  8  9  8  8  8]]
F0= [[-233.   24. -201.   35.   24.  -12.  -16.   10.]
 [-124.  -14.    7.  -70.   44.  -30.  -25.    8.]
 [   0.  -28.   43.    4.   -3.    0.  -12.   16.]
 [   6.   -8.   12.   -4.   -8.  -21.   72.  -40.]
 [ -12.   12.   24.  -12.  -10.   18.  -16.    0.]
 [ -10.    6.  -12.    0.   12.  -24.  -27.   28.]
 [  -8.   10.    6.    7.  -16.  -10.    0.   -8.]
 [   6.    0.   -8.    0.   -9.   -8.   24.    0.]]


# V. TRANSFORMÉE EN COSINUS DISCRÈTE INVERSE
La formule permettant de calculer la DCT inverse est donnée :

![title](images/dct.png)


13. **Toujours dans le même fichier (cf Figure 8), compléter la fonction suivante tcdi(F0) à
l’aide de la formule ci-dessus (π = math.pi) :**


![title](images/fig8.png)

14. **Executer la fonction pour afficher la matrice S et la stocker dans une variable :**


In [9]:
import importlib # Faire qu’une fois
importlib.reload(extract_viterbi2)   # Refaire apres chaque modification
S = extract_viterbi.tcdi(F0)


[ 64   1 222 244  25  97 193 161 186 197  72 119  53  11 115]
S= [[-85. -52. -46. -36. -18. -64. -79. -41.]
 [-77. -66. -45.   1.   1. -54. -58. -51.]
 [-80. -42. -35.  -3.  32. -56. -60. -89.]
 [-63. -34. -37.  14.  29. -70. -44. -94.]
 [-40. -43.   4.  19. -10.  23. -65. -68.]
 [-32. -30.  -8.   9.   9.  45. -61. -63.]
 [-27. -33.  -4.   5.  53.  24. -27. -63.]
 [ -9. -17. -19.   4.  16.   4.  22. -78.]]


15. **Pourquoi a t-on des nombres négatifs ? Que reste t-il à faire ? Le réaliser.**


In [10]:
S=S+128
S

array([[ 43.,  76.,  82.,  92., 110.,  64.,  49.,  87.],
       [ 51.,  62.,  83., 129., 129.,  74.,  70.,  77.],
       [ 48.,  86.,  93., 125., 160.,  72.,  68.,  39.],
       [ 65.,  94.,  91., 142., 157.,  58.,  84.,  34.],
       [ 88.,  85., 132., 147., 118., 151.,  63.,  60.],
       [ 96.,  98., 120., 137., 137., 173.,  67.,  65.],
       [101.,  95., 124., 133., 181., 152., 101.,  65.],
       [119., 111., 109., 132., 144., 132., 150.,  50.]])

# VI. ENFIN DES IMAGES !
On utilise le module image de PIL :


In [11]:
import PIL
image = PIL.Image.fromarray(S)
image.show()

16. **Qu’observez-vous ?**


Pas grand chose... Un petit bout de l'image 8x8 pixels..

On va essayer de faire mieux en affichant les 14 imagettes de la ligne 25 :

17. **En récapitulant toutes les étapes précédentes dans une boucle, remplir une liste s=[]
contenant les 14 matrices S. Puis à l’aide de la fonction np.concatenate((a,b), axis=1)
rassembler toutes les matrices en une seule ligne et afficher l’image.**


In [12]:
s = []
for i in range(14):
    DCT = extract_viterbi.zigzag(26,14,i)
    F0 = extract_viterbi.quantif(DCT,96)
    S = extract_viterbi.tcdi(F0)
    S = S +128
    s.append(S)

octets= [248  91  88 248   7 246  31 216 219 196 115 232 115  89 219 248 175 195 151 145 234 205  52 209  43  92  40 134  54 139 106  35  28 237  69 105  47  25 136 218 139 111 176   1 230   7 168  53  95 217 243 246 128 248 163 162 252  56 139 226  79 197 184 254  34 233  55  51 201 118 116 171 219  51  45 189 165 218 219 198  17]

nbr total octets compressés= 740
dct= [-233, 24, -124, 0, -14, -201, 35, 7, -28, 6, -12, -8, 43, -35, 12, -4, 22, 2, 6, 6, -5, -2, 2, 8, -2, -1, -6, -4, 2, -5, 0, -2, -3, -3, 2, 1, 0, 1, 0, -2, -3, -2, 2, 4, 12, 2, 2, 1, -1, 0, -2, -3, -2, -8, 0, -3, -1, -1, -1, 0, 4, -1, 3]
2
dct= [-7, 21, -131, 128, 81, -26, -71, -5, 73, 91, -4, 69, -22, 9, 7, 4, 11, 1, -21, 13, -12, 1, 5, -8, -11, 11, -2, -8, -4, -1, -3, -2, -2, 3, -5, 3, -1, -2, 0, 0, 3, -1, 7, 10, -4, -1, 2, -1, 1, -2, -1, 0, -1, -3, -3, 1, -2, 1, 0, 0, 3, -1, 1]
3
dct= [111, -43, -102, -61, 50, -12, 3, -26, 27, -88, -29, -52, -5, -46, -1, 7, -37, 6, -5, -14, 19, 1, 3, 0, 10, 11, -4, 9, 4, -2, 6, 9, -2

In [13]:
import numpy as np
S=s[0]
for i in range(len(s)-1):
    S = np.concatenate((S,s[i+1]), axis=1)
image = PIL.Image.fromarray(S)
image.show()

18. **Reconnaissez vous cet endroit sur l’image entière ? Enregistrer l’image au format bmp
(clic - File - Save) et à l’aide de The Gimp par exemple, superposez cette imagette sur
l’image originale pour montrer la correspondance.**

![title](images/fin.png)


## Références :

[1] J.-M Friedt, Décodage d’images numériques issues de satellites météorologiques en orbite basse :
le protocole LRPT de Meteor-M2 (partie 1/2), janvier 2019, http ://jmfriedt.free.fr/glmf_meteor1.pdf

[2] J.-M Friedt, Décodage d’images numériques issues de satellites météorologiques en orbite basse :
le protocole LRPT de Meteor-M2 (partie 2/2), mars 2019, http ://jmfriedt.free.fr/glmf_meteor1.pdf

[3] Memon Nasir, ... Ansari Rashid, in Handbook of Image and Video Processing (Second Edition),
2005, The JPEG Lossless Image Compression Standards, https ://www.sciencedirect.com/topics/engineering/huffm
table

